# A custom spelling prediction endpoint



[Peter Parente](https://github.com/parente) has a great post on creating [IPython notebook & server extensions.](https://mindtrove.info/4-ways-to-extend-jupyter-notebook/#nb-server-exts)  The documentation on [Custom Request Handlers](https://jupyter-notebook.readthedocs.io/en/stable/extending/handlers.html) is another important reference. We will use this post
as a foundation for creating the `spacy` endpoint.

## Configuring the endpoint

Our extensions module will be __deathbeds.spacy_endpoint__.  The configuration has change from `c.NotebookApp.server_extensions`

In [3]:
    %%file C:/Users/deathbeds/.jupyter/jupyter_notebook_config.py
    c.NotebookApp.server_extensions = [
        'deathbeds.spacy_endpoint'
    ]

Overwriting C:/Users/deathbeds/.jupyter/jupyter_notebook_config.py


to `c.NotebookApp.nbserver_extensions` which is a dictionary indicating whether an extension is enabled.

In [4]:
    %%file C:/Users/deathbeds/.jupyter/jupyter_notebook_config.py
    c.NotebookApp.nbserver_extensions = {
        'deathbeds.spacy_endpoint': True
    }

Overwriting C:/Users/deathbeds/.jupyter/jupyter_notebook_config.py


## The completer

We have __2__ completer examples to work off of: [Completer for indented code](http://nbviewer.jupyter.org/github/deathbeds/deathbeds.github.io/blob/master/deathbeds/2018-07-03-Custom-IPython-Completer-for-Indented-Code.ipynb) & [Word completion with spacy](http://nbviewer.jupyter.org/github/deathbeds/deathbeds.github.io/blob/master/deathbeds/2018-09-04-Word-Completer.ipynb).  We have already tackled parts of the problem at hand, but a shortcoming of the past approach was the time taken to load the corpus.  An extension will require the corpus loaded just once. 

In [5]:
    import requests, ast

In [6]:
    def event(self, event):
        return ast.literal_eval(requests.get('http://localhost:8888/spacy', params={
            'complete': event.symbol
        }).text)


In [7]:
    def load_ipython_extension(ip):
        ip.set_hook('complete_command', event, re_key=".*")
        
    if __name__ == '__main__': load_ipython_extension(get_ipython())

## Building the endpoint

### Rely on the `spacy.vocab` for the words to complete.

In [8]:
    %%file spacy_endpoint.py
    from notebook.utils import url_path_join
    from notebook.base.handlers import IPythonHandler
    import spacy

    en = spacy.load('en')
    words = list(x.text for x in en.vocab)
    del en
    del spacy

Overwriting spacy_endpoint.py


## The endpoint

* [Consuming `tornado` query parameters.](https://stackoverflow.com/questions/10726486/tornado-url-query-parameters)

In [7]:
    %%file spacy_endpoint.py -a
    
    class CompleteHandler(IPythonHandler):
        def get(self): self.finish(str(list(x for x in words if x.startswith(
            self.request.arguments['complete'][0].decode('utf-8')))))
    

Appending to spacy_endpoint.py


## The extension

The `load_jupyter_server_extension` is required to make sure the Jupyter and IPython mount the extension.

In [9]:
    %%file spacy_endpoint.py -a
    
    def load_jupyter_server_extension(nb_app):
        web_app = nb_app.web_app
        host_pattern = '.*$'
        route_pattern = url_path_join(web_app.settings['base_url'], '/spacy')
        web_app.add_handlers(host_pattern, [(route_pattern, CompleteHandler)])

Appending to spacy_endpoint.py


## Outcome

The next time `jupyter` is run an endpoint for `spacy` with exist.